**Импорты**

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from tqdm.auto import tqdm

from ignite.engine import create_supervised_evaluator, create_supervised_trainer, Events
from ignite.metrics import Accuracy, Loss
from ignite.utils import setup_logger

In [ ]:
def run(epochs, lr, log_interval):
    model = Torchic()
    device = 'cpu'
    if torch.cuda.is_available():
        device = 'cuda'

    optimizer = opt.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    trainer = create_supervised_trainer(model, optimizer, criterion, device=device)
    trainer.logger = setup_logger('trainer')

    val_metrics = {'accuracy': Accuracy(), 'cre': Loss(criterion)}
    evaluator = create_supervised_evaluator(model, metrics=val_metrics, device=device)
    evaluator.logger = setup_logger('evaluator')

    pbar = tqdm(initial=0, leave=False, total=len(train_dataloader), desc=f"ITERATION - loss: {0:.2f}")

    @trainer.on(Events.ITERATION_COMPLETED(every=log_interval))
    def log_training_loss(engine):
        pbar.desc = f'Iteration - loss: {engine.state.output:.2f}'
        pbar.update(log_interval)

    @trainer.on(Events.EPOCH_COMPLETED)
    def log_training_results(engine):
        evaluator.run(val_dataloader)
        metrics = evaluator.state.metrics
        avg_accuracy = metrics['accuracy']
        avg_cre = metrics['cre']
        tqdm.write(
            f"Validation Results - Epoch: {engine.state.epoch} Avg accuracy: {avg_accuracy:.2f} Avg loss: {avg_cre:.2f}"
        )
        pbar.n = pbar.last_print_n = 0

    @trainer.on(Events.EPOCH_COMPLETED | Events.COMPLETED)
    def log_time(engine):
        tqdm.write(f"{trainer.last_event_name.name} took { trainer.state.times[trainer.last_event_name.name]} seconds")

    trainer.run(train_dataloader, max_epochs=epochs)
    pbar.close()

In [ ]:
run(epochs=10, lr=0.001, log_interval=10)